A review of the LIKE operator

In [ ]:
-- Select all columns
SELECT *
FROM film
-- Select only records that begin with the word 'GOLD'
WHERE title LIKE 'GOLD%';

In [ ]:
SELECT *
FROM film
-- Select only records that end with the word 'GOLD'
WHERE title LIKE '%GOLD';

In [ ]:
SELECT *
FROM film
-- Select only records that contain the word 'GOLD'
WHERE title LIKE '%GOLD%';

A basic full-text search

In [ ]:
-- Select the film description as a tsvector
SELECT to_tsvector(description)
FROM film;

* Select the title and description columns from the film table.
* Perform a full-text search on the title column for the word elf.

In [ ]:
-- Select the title and description
SELECT title, description
FROM film
-- Convert the title to a tsvector and match it against the tsquery 
WHERE to_tsvector(title) @@ to_tsquery('elf');

User-defined data types

In [ ]:
-- Create an enumerated data type, compass_position
CREATE TYPE compass_position AS ENUM (
  	-- Use the four cardinal directions
  	'North', 
  	'South',
  	'East', 
  	'West'
);
-- Confirm the new data type is in the pg_type system table
SELECT *
FROM pg_type
WHERE typname='compass_position';

Getting info about user-defined data types

* Select the column_name, data_type, udt_name.
* Filter for the rating column in the film table

In [ ]:
-- Select the column name, data type and udt name columns
SELECT column_name, data_type, udt_name
FROM INFORMATION_SCHEMA.COLUMNS 
-- Filter by the rating column in the film table
WHERE table_name ='film' AND column_name='rating';

In [ ]:
SELECT *
FROM pg_type 
WHERE typname='mpaa_rating'

In [ ]:
-- Select the film title and inventory ids
SELECT 
	f.title, 
    i.inventory_id
FROM film as f 
	-- Join the film table to the inventory table
	INNER JOIN inventory AS i ON f.film_id=i.film_id 

In [ ]:
-- Select the film title and inventory ids
SELECT 
	f.title, 
    i.inventory_id,
    -- Determine whether the inventory is held by a customer
    inventory_held_by_customer(i.inventory_id) AS held_by_cust
FROM film as f 
	INNER JOIN inventory AS i ON f.film_id=i.film_id 

In [ ]:
-- Select the film title and inventory ids
SELECT 
	f.title, 
    i.inventory_id,
    -- Determine whether the inventory is held by a customer
    inventory_held_by_customer(i.inventory_id) as held_by_cust
FROM film as f 
	INNER JOIN inventory AS i ON f.film_id=i.film_id 
WHERE
	-- Only include results where the held_by_cust is not null
    inventory_held_by_customer(i.inventory_id) IS NOT NULL

Enabling extensions in postgres

In [ ]:
-- Enable the pg_trgm extension
CREATE EXTENSION IF NOT EXISTS pg_trgm;

Now confirm that both fuzzystrmatch and pg_trgm are enabled by selecting all rows from the appropriate system table.

In [ ]:
-- Select all rows extensions
SELECT * 
FROM pg_extension;

Measuring similarity between two strings

In [ ]:
-- Select the film title and description.
-- Calculate the similarity between the title and description.

-- Select the title and description columns
SELECT 
  title, 
  description, 
  -- Calculate the similarity
  similarity(title, description)
FROM 
  film

Levenshtein distance examples

In [ ]:
-- Select the film title and film description.
-- Calculate the levenshtein distance for the film title with the string JET NEIGHBOR.

-- Select the title and description columns
SELECT  
  title, 
  description, 
  -- Calculate the levenshtein distance
  levenshtein(title, 'JET NEIGHBOR') AS distance
FROM 
  film
ORDER BY 3

Putting it all together

In [ ]:
-- ! Step 1:
-- Select the title and description for all DVDs from the film table.
-- Perform a full-text search by converting the description to a tsvector
-- ...and match it to the phrase 'Astounding & Drama' using a tsquery in the WHERE clause.

-- Select the title and description columns
SELECT  
  title, 
  description 
FROM 
  film 
WHERE 
  -- Match "Astounding Drama" in the description
  to_tsvector(description) @@ 
  to_tsquery('Astounding & Drama');

In [ ]:
-- ! Step 2:

-- Add a new column that calculates the similarity of the description
-- ...with the phrase 'Astounding Drama'.
-- Sort the results by the new similarity column in descending order.

SELECT 
  title, 
  description, 
  -- Calculate the similarity
  similarity(description, 'Astounding Drama') 
FROM 
  film 
WHERE 
  to_tsvector(description) @@ 
  to_tsquery('Astounding & Drama') 
ORDER BY 
	similarity(description, 'Astounding Drama') DESC;